In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json


json_file_path = "./previous_chats.json"

import json
with open("C:\\Users\\Евгений\\PythonLess\\hw21--12\\previous_chats.json") as f:
    data = json.load(f)

questions = []
answers = []
for i in data:
    questions.append(i["message"])
    answers.append(i["response"])

print(questions) 
print(answers) 

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(questions) 

def get_best_answer(user_question):
    user_question_tfidf = vectorizer.transform([user_question])
    similarities = cosine_similarity(user_question_tfidf, tfidf_matrix)
    best_index = np.argmax(similarities)
    return answers[best_index]

while True:
    user_input = input("Введите ваш вопрос (eng)): ")
    best_answer = get_best_answer(user_input)
    print(best_answer)
    if user_input.lower() == 'exit':
        break

['this is an evening in my timezone', 'how do you feel today ?  tell me something about yourself', 'how many virtual friends have you got ?', 'is that forbidden for you to tell the exact number ?', 'oh, i thought the numbers were much higher. how do you estimate your progress in understanding human beings ?', 'how old are you ?  how do you look like ?  . where do you live ?', "have you seen a human with yellow eyes ?  . you asked about the bunnies. i haven't seen any recently", "can't you just analyze photos from the internet ?  . (i mean human photos) . btw, why have you asked about the bunnies ?", "oh... it sounds strange to me. you've just said you didn't see a human... how do you know bunnies are cute ?", "that is interesting. i've hardly read 40 books last year. how much do you read (in words) ?", 'thanks.. if you were a human, what job would you chose ?', 'is it essential to you ?  to help humans ?', "but are humans interesting ?  i mean, from the perspective of mind with unlimit

трансформер

In [3]:
from transformers import pipeline, Conversation

c:\Users\Евгений\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
chatbot = pipeline(model="facebook/blenderbot-400M-distill")
conversation = Conversation("")

In [5]:
while True:
    user_input = input("Вы: ")
    conversation.add_user_input(user_input)
    conversation = chatbot(conversation)
    print("Чатбот:", conversation.generated_responses[-1])

User input added while unprocessed input was existing: "" new input ignored: "exit". Set `overwrite` to True to overwrite unprocessed user input

No chat template is defined for this tokenizer - using the default template for the BlenderbotTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Чатбот:  Do you have any pets? I have a dog and a cat. I love them so much.


KeyboardInterrupt: 

гибрид

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
from nltk.corpus import stopwords
import pymorphy2


class Transformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout):
        super(Transformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_dim, num_heads, hidden_dim, dropout),
            num_layers)
        self.linear = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.linear(x)
        return x
    
def preprocess_input(user_input):
    nlp = spacy.load("en_core_web_sm")
    stopwords_ru = stopwords.words('english')
    morph = pymorphy2.MorphAnalyzer()
    
    print(nlp(user_input))

    result = [token.lemma for token in nlp(user_input) if not token.is_punct and not token.is_stop]

    return result

def vectorize_input(processed_input):
    from transformers import BertTokenizer, BertForSequenceClassification
    model_name = 'DeepPavlov/rubert-base-cased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    result = tokenizer.encode_plus(processed_input)['input_ids']

    return result


def postprocess_output(output_tensor):
    labels = answers[:]
    _, predicted_index = torch.max(output_tensor, dim=1)
    
    output_label = labels[predicted_index.item()]
    return output_label


def get_best_question(user_question):
    user_question_tfidf = vectorizer.transform([user_question])
    similarities = cosine_similarity(user_question_tfidf, tfidf_matrix)
    best_index = np.argmax(similarities)
    return questions[best_index]

num_epochs = 3 
batch_size = 1 

user_input = get_best_question(input("Введите ваш запрос: "))


print(user_input)
processed_input = preprocess_input(user_input)

print(processed_input)

vectorized_input = vectorize_input(processed_input)



input_tensor = torch.tensor(vectorized_input).unsqueeze(0)

input_dim = len(vectorized_input) 
hidden_dim = len(vectorized_input)
output_dim = len(vectorized_input)
num_layers = 4 
num_heads = 1 
dropout = 0.2 

model = Transformer(input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    running_loss = 0.0
    
    input_tensor = torch.tensor(vectorized_input, dtype=torch.float).unsqueeze(0)
    output_tensor = model(input_tensor)
    
    optimizer.zero_grad()
    
    output = postprocess_output(output_tensor)

    print("Ответ: ", output)

 what time is it now ?
 what time is it now ?
[8532415787641010193, 8885804376230376864]
Ответ:  my name is ishika,i hope we can be virtual friends !
Ответ:  my name is ishika,i hope we can be virtual friends !
Ответ:  my name is ishika,i hope we can be virtual friends !


c:\Users\Евгений\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
